In [1]:
import json
import re

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from textblob import TextBlob

In [2]:
with open('dataset/goodreads_reviews_spoiler.json') as json_file:
    data = json.loads('[' + ',\n'.join(json_file.readlines()) + ']')

# Preprocess

In [3]:
def unpack(sents):
    ys = []
    xs = []
    for y, sent in sents:
        ys.append(y)
        xs.append(sent)
    return ys, xs

In [4]:
x = []
y = []
for sample in tqdm(data):
    ans, sent = unpack(sample['review_sentences'])
    x.extend(sent)
    y.extend(ans)

100%|██████████| 1378033/1378033 [00:03<00:00, 405316.66it/s]


In [5]:
del data

In [6]:
for id, i in tqdm(enumerate(x), total=len(x)):
    text = re.sub(r'http\S+', '', i.lower())
    # text = str(TextBlob(text).correct())
    x[id] = text

100%|██████████| 17672655/17672655 [00:29<00:00, 601761.71it/s]


In [7]:
prep_data = pd.DataFrame([x,y], index=['text', 'target']).T

In [8]:
prep_data = prep_data[~(prep_data['text'] == '')]

In [34]:
prep_data

,text,target
0,this is a special book.,0
1,"it started slow for about the first third, the...",0
2,this is what i love about good science fiction...,0
3,"it is a 2015 hugo winner, and translated from ...",0
4,for instance the intermixing of chinese revolu...,0
...,...,...
17672650,"""up until then it's a good day--one of the best.",0
17672651,"close to perfect, really, even though nothing ...",0
17672652,i guess i've probably had a lot of days like t...,0
17672653,"perhaps you to have a thousands tomorrow's, bu...",0


In [40]:
prep_data = prep_data[prep_data.apply(lambda x: len(x[0]) <= 500, axis=1)]

In [41]:
prep_data

,text,target
0,this is a special book.,0
1,"it started slow for about the first third, the...",0
2,this is what i love about good science fiction...,0
3,"it is a 2015 hugo winner, and translated from ...",0
4,for instance the intermixing of chinese revolu...,0
...,...,...
17672650,"""up until then it's a good day--one of the best.",0
17672651,"close to perfect, really, even though nothing ...",0
17672652,i guess i've probably had a lot of days like t...,0
17672653,"perhaps you to have a thousands tomorrow's, bu...",0


In [46]:
(17659023 - len(prep_data))/17659023

6.466948935963219e-05

In [47]:
prep_data['text'] = prep_data['text'].apply(lambda x: np.str_(x))

/tmp/ipykernel_4820/2054751173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_data['text'] = prep_data['text'].apply(lambda x: np.str_(x))


In [48]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(prep_data, test_size=0.2)

In [49]:
train.to_csv('dataset/baseline/train.csv', index=False)
test.to_csv('dataset/baseline/test.csv', index=False)

# TF-IDF baseline

In [1]:
import text_clf

In [2]:
CONFIG = 'config/baseline_config.yaml'

In [3]:
model, target_names_mapping = text_clf.train(path_to_config=CONFIG)

text-clf-train - INFO - Config:

seed: 42
path_to_save_folder: models
experiment_name: model

# data
data:
  train_data_path: dataset/baseline/train.csv
  test_data_path: dataset/baseline/test.csv
  sep: ','
  text_column: text
  target_column: target

# preprocessing
# (included in resulting model pipeline, so preserved for inference)
preprocessing:
  lemmatization: null  # pymorphy2

# tf-idf
tf-idf:
  lowercase: true
  ngram_range: (1, 1)
  max_df: 1.0
  min_df: 1

# logreg
logreg:
  penalty: l2
  C: 1.0
  class_weight: balanced
  solver: saga
  n_jobs: -1

# grid-search
grid-search:
  do_grid_search: false
  grid_search_params_path: hyperparams.py
text-clf-train - INFO - Loading data...
text-clf-train - INFO - Train dataset size: 14126304
text-clf-train - INFO - Test dataset size: 3531577
text-clf-train - INFO - Fitting TF-IDF + LogReg model...
[Pipeline] ............ (step 1 of 2) Processing tf-idf, total= 4.3min
[Pipeline] ............ (step 2 of 2) Processing logreg, total=23.4m

/home/xx/anaconda3/envs/diploma/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


text-clf-train - INFO - Train classification report:

              precision    recall  f1-score   support

           0       0.98      0.77      0.86  13670521
           1       0.07      0.53      0.13    455783

    accuracy                           0.76  14126304
   macro avg       0.53      0.65      0.49  14126304
weighted avg       0.95      0.76      0.84  14126304

text-clf-train - INFO - Train confusion matrix:

          0        1
0  10527985  3142536
1    213990   241793

text-clf-train - INFO - Test classification report:

              precision    recall  f1-score   support

           0       0.98      0.77      0.86   3417771
           1       0.07      0.50      0.12    113806

    accuracy                           0.76   3531577
   macro avg       0.52      0.63      0.49   3531577
weighted avg       0.95      0.76      0.84   3531577

text-clf-train - INFO - Test confusion matrix:

         0       1
0  2627970  789801
1    56895   56911

text-clf-train - INF